TODO

In [ ]:
import os
import os.path
import re

import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from IPython.display import display

Some settings

In [ ]:
plt.rcParams['figure.figsize'] = [20, 8]
topleveldir = os.path.join(os.getcwd(), '..', 'results', '20220707saparamsearch')

Parameters

In [ ]:
# Select from: 15, 30, 45, 60, 75
temperatures = [15]
# Select from 0.95, 0.99
alfas = [0.95]
# Select from 2, 4, 6, 8, 10
multipliers = [2, 4]
# select from 0: (1, 0, 0); 1: (.9, .1, 0); 2: (.9, .05, .05); 3: (.75, .25, 0); 4: (.75, .15, .1)
fracs = [0, 1, 2, 3, 4]
# select from 5, 10, 15, 20, 50
ns = [50]
# select from 25.00, 50.00, 100.00, 200.00
ps = [50.00]

# Compile sets of parameters
param_sets = [(temp, alfa, mult, frac, n, p) for temp in temperatures for alfa in alfas for mult in multipliers for frac in fracs for n in ns for p in ps]

Utility functions for getting the summary & iterations data files for given parameters

In [ ]:
def summary_files(basepath, temp, alfa, mult, frac):
    """Returns a list of all summary files associated with the parameter values."""
    summary_files = []
    summary_dir = os.path.join(basepath, f'temp{temp}alfa{alfa:.2f}mult{mult}frac{frac}')
    pattern = re.compile('[0-9_]*_summary.csv$')
    for root, directories, files in os.walk(summary_dir):
        for file in files:
            if pattern.match(file):
                summary_files.append(os.path.join(root, file))
    return summary_files

def iterations_file(basepath, temp, alfa, mult, frac, n, p):
    """Returns the path to the directory associated with the parameter values."""
    report_dir = ""
    top_lvl_dir = os.path.join(basepath, f'temp{temp}alfa{alfa:.2f}mult{mult}frac{frac}')
    pattern = re.compile('[0-9]*_n' + re.escape(f'{n}') + 'r' + re.escape(f'{p:.2f}') + 'a0i0$')
    for root, directories, files in os.walk(top_lvl_dir):
        for directory in directories:
            if pattern.match(directory):
                report_dir = os.path.join(root, directory)
    return os.path.join(report_dir, "iterations.csv")

Define function for displaying a progress graph for the selected parameters

In [ ]:
def display_iteration_graph(sets, iter_limit=None):
    fig, (ax1, ax2) = plt.subplots(2, 1)
    ax1.set_xlabel('Number of iterations')
    ax2.set_xlabel('Number of iterations')
    ax1.set_ylabel('Objective value (best)')
    ax2.set_ylabel('Objective value (current)')
    ax1.set_title('Evolution of the value of the best found solution')
    ax2.set_title('Evolution of the value of the current solution')
    if iter_limit is not None:
        ax1.set_xlim(0, iter_limit)
        ax2.set_xlim(0, iter_limit)
    for param_set in sets:
        iter_path = iterations_file(topleveldir, param_set[0], param_set[1], param_set[2], param_set[3], param_set[4], param_set[5])
        data = pd.read_csv(
            iter_path,
            sep=';',
            usecols=([
                '#', 'time', 'best_score', 'curr_score', 'rejected', 'slack'
            ])
        )
        ax1.plot(
            data['#'],
            data['best_score'],
            label=f"({param_set[0]}, {param_set[1]:.2f}, {param_set[2]}, {param_set[3]}, {param_set[4]}, {param_set[5]:.2f})"
        )
        ax2.plot(
            data['#'],
            data['curr_score'],
            label=f"({param_set[0]}, {param_set[1]:.2f}, {param_set[2]}, {param_set[3]}, {param_set[4]}, {param_set[5]:.2f})"
        )
    handles, labels = ax1.get_legend_handles_labels()
    fig.legend(handles, labels, loc="upper center")

Define function for collecting some results in a table

In [ ]:
def display_summary_results(sets):
    collect_data = []
    for param_set in sets:
        summ_files = summary_files(topleveldir, param_set[0], param_set[1], param_set[2], param_set[3])
        for potential_summary in summ_files:
            data = pd.read_csv(
                potential_summary,
                sep=';',
                usecols=([
                    'n', 'r', '#iter', 'time', 'best', 'slack'
                ])
            )
            pot_data = data.loc[(data['n'] == param_set[4]) & (data['r'] == param_set[5])]
            if pot_data.shape[0] > 0:
                collect_data.append(pot_data.values.tolist()[0])
                break
    df = pd.DataFrame(collect_data, columns=['n', 'r', '#iter', 'time', 'best', 'slack'])
    display(df)
            

Display summary statistics

In [ ]:
display_iteration_graph(param_sets, iter_limit=None)
display_summary_results(param_sets)